In [23]:
import pandas as pd
import numpy as np
import editdistance

from pymystem3 import Mystem
import json

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import accuracy_score

from gensim.models import KeyedVectors
from sklearn.model_selection import GridSearchCV

/home/vera/.venv/py3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/vera/.venv/py3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [72]:
stemmer = Mystem()

mystem_to_upos = {
    "A": "ADJ",
    "ADV": "ADV",
    "ADVPRO": "ADV",
    "ANUM": "ADJ",
    "APRO": "DET",
    "COM": "ADJ",
    "CONJ": "SCONJ",
    "INTJ": "INTJ",
    "NONLEX": "X",
    "NUM": "NUM",
    "PART": "PART",
    "PR": "ADP",
    "S": "NOUN",
    "SPRO": "PRON",
    "UNKN": "X",
    "V": "VERB",
}

def tag(word, tagset=None):
    """Get lexema from word and add tag"""
    if not tagset:
        return word.lower()
        
    processed = stemmer.analyze(word)[0]
    
    try:
        lemma = processed["analysis"][0]["lex"].lower().strip()

        pos = processed["analysis"][0]["gr"].split(',')[0]
        pos = pos.split('=')[0].strip()

        if tagset == Tagset.UPOS:
            return "{}_{}".format(lemma, mystem_to_upos[pos])
        else:
            return "{}_{}".format(lemma, pos)
    except:
        return "word"

tag("красивый", Tagset.UPOS)

'красивый_ADJ'

In [78]:
from enum import Enum
class Tagset(Enum):
    UPOS = 1
    MYSTEM = 2

class ModelType(Enum):
    W2V = 1
    FASTTEXT = 2

folder_path = '/mnt/data/embedding_models/'
config = [
    {'path': folder_path+'araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model', 'type': ModelType.FASTTEXT, 'tagset': None},
    {'path': folder_path+'araneum_none_fasttextskipgram_300_5_2018/araneum_none_fasttextskipgram_300_5_2018.model', 'type': ModelType.FASTTEXT, 'tagset': None},
    {'path': folder_path+'araneum_upos_skipgram_300_2_2018.vec', 'type': ModelType.W2V, 'tagset': Tagset.UPOS},
    {'path': folder_path+'araneum_upos_skipgram_600_2_2017.bin', 'type': ModelType.W2V, 'tagset': Tagset.UPOS},
    {'path': folder_path+'news_mystem_skipgram_1000_20_2015.bin', 'type': ModelType.W2V, 'tagset': Tagset.MYSTEM},
    {'path': folder_path+'news_upos_cbow_300_2_2017.bin', 'type': ModelType.W2V, 'tagset': Tagset.UPOS},
    {'path': folder_path+'news_upos_cbow_600_2_2018.vec', 'type': ModelType.W2V, 'tagset': Tagset.UPOS},
    {'path': folder_path+'ruscorpora_mystem_cbow_300_2_2015.bin', 'type': ModelType.W2V, 'tagset': Tagset.MYSTEM}, 
    {'path': folder_path+'ruscorpora_upos_skipgram_300_10_2017.bin', 'type': ModelType.W2V, 'tagset': Tagset.UPOS},
    {'path': folder_path+'ruscorpora_upos_skipgram_300_5_2018.vec', 'type': ModelType.W2V, 'tagset': Tagset.UPOS},
    {'path': folder_path+'ruscorpora_upos_skipgram_600_10_2017.bin', 'type': ModelType.W2V, 'tagset': Tagset.UPOS},
    {'path': folder_path+'ruwikiruscorpora-nobigrams_upos_skipgram_300_5_2018.vec', 'type': ModelType.W2V, 'tagset': Tagset.UPOS},
    {'path': folder_path+'ruwikiruscorpora-superbigrams_skipgram_300_2_2018.vec', 'type': ModelType.W2V, 'tagset': Tagset.UPOS},
    {'path': folder_path+'ruwikiruscorpora_upos_cbow_300_20_2017.bin', 'type': ModelType.W2V, 'tagset': Tagset.UPOS},
    {'path': folder_path+'ruwikiruscorpora_upos_skipgram_300_2_2018.vec', 'type': ModelType.W2V, 'tagset': Tagset.UPOS},
    {'path': folder_path+'taiga_upos_skipgram_300_2_2018.vec', 'type': ModelType.W2V, 'tagset': Tagset.UPOS},
    {'path': folder_path+'web_mystem_skipgram_500_2_2015.bin', 'type': ModelType.W2V, 'tagset': Tagset.MYSTEM},
    {'path': folder_path+'web_upos_cbow_300_20_2017.bin', 'type': ModelType.W2V, 'tagset': Tagset.UPOS},
    
    
]
# ruwikiruscorpora_mystem_cbow_500_2_2015.bin.gz - damaged, can't unpack

In [63]:
from gensim.models import FastText
# model = FastText.load_fasttext_format('/mnt/data/embedding_models/araneum_none_fasttextcbow_300_5_2018.model',encoding='utf8')
model = FastText.load('/mnt/data/embedding_models/araneum_none_fasttextskipgram_300_5_2018.model')

print(model['резюме'])

[ 7.48695880e-02 -1.52753353e-01  1.06077110e-04 -1.74655825e-01
 -5.14660291e-02 -1.89847872e-01 -2.61208322e-02 -1.18963845e-01
 -2.08503697e-02 -2.95836300e-01 -3.31608772e-01  2.70554423e-01
 -1.76262170e-01 -2.23780394e-01  1.69155806e-01 -6.61839694e-02
  1.13103688e-02 -1.03949293e-01  2.36681402e-02  1.03438698e-01
 -6.01457357e-02 -6.38513640e-02  5.31587183e-01 -1.37142286e-01
  2.87178196e-02  9.94504243e-02  1.16282701e-01  2.87400156e-01
 -4.73069847e-01 -2.89020360e-01  3.42368707e-02 -4.85637844e-01
  1.73965484e-01  1.62362844e-01  3.82165164e-01 -8.92007202e-02
 -2.17388257e-01  5.72458446e-01  1.03656717e-01  2.65044957e-01
  1.24716468e-01 -3.65553677e-01 -2.65571684e-01  8.31584871e-01
 -9.67214927e-02 -4.42985818e-02  2.39112809e-01 -9.13406461e-02
 -1.37247711e-01  4.71524661e-03  7.52480850e-02 -4.68000732e-02
  1.80732831e-01 -9.50379483e-03 -2.13996440e-01 -5.52565381e-02
 -4.05137539e-02  3.33702028e-01 -5.52371621e-01  9.02125239e-03
 -3.11136246e-04  7.34406

/home/vera/.venv/py3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [64]:
# DIM = len(model.words)

DIM = len(model[next(iter(model.wv.vocab.keys()))])
print(DIM)

300


/home/vera/.venv/py3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
def get_words(text):
    return " ".join([
        tag(word) for word in text.split()
    ])

In [21]:
def calculate_vector(text, model=model):                                          
    vector = np.zeros(DIM)
    words = [tag(word) for word in text.split()]

    words_in_model = 0                                                  
    for word in words:                                               
        if word in model.wv.vocab:                                              
            vector += model[word]                           
            words_in_model += 1                                             

    if words_in_model > 0:                                                  
        vector /= words_in_model                                            

    return vector 

In [65]:
with open("/home/vera/projects/profile-identity/data/dialog-ru-ru.json", "r") as infile:
    data = json.load(infile)
    
X, Y = [], []

for label, category in enumerate(data):
    for q in category["questions"]:
        Y.append(label)
        X.append(calculate_vector(q))
        
X = np.array(X)
Y = np.array(Y)

/home/vera/.venv/py3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [30]:
clf = KNeighborsClassifier(n_neighbors=3, metric="manhattan")

In [31]:
loo = LeaveOneOut()

In [67]:
y_true, y_pred = [], []

parameters = {'metric':('manhattan', 'euclidean', 'chebyshev',
                        'minkowski'), 'n_neighbors':[2, 3, 5, 10, 20]}
grd = GridSearchCV(clf, parameters, cv = loo)
grd.fit(X, Y)
print(grd.best_score_)
print(grd.best_params_)

# for train_index, test_index in loo.split(X):
#     X_train, X_test = X[train_index], X[test_index]
#     Y_train, Y_test = Y[train_index], Y[test_index]
    
#     grd.fit(X_train, Y_train)
    
#     Y_predict = grd.predict(X_test)
    
#     y_true += list(Y_test)
#     y_pred += list(Y_predict)
#     print(grd.best_params_)
# print(accuracy_score(y_true, y_pred))


0.38095238095238093
{'metric': 'manhattan', 'n_neighbors': 2}


In [41]:
if 'ракетостроение' in model.wv.vocab:
    print('true')


true
